In [ ]:
from utils import img_slices_coords, rotate_normalized_coords, calculate_overlap

In [ ]:
from PIL import Image

In [19]:
labels = {
    "圆形": "circle",
    "未熔合": "lack_of_fusion",
    "未焊透": "lack_of_penetration",
    "条形": "line",
    "内凹": "pit",
    "咬边": "undercut",
}

In [20]:
labels = list(labels.values())

In [21]:
labels

['circle', 'lack_of_fusion', 'lack_of_penetration', 'line', 'pit', 'undercut']

In [ ]:
from pathlib import Path
root_path = Path("dataset")
train_origin_path = root_path / "train-origin"
valid_origin_path = root_path / "valid-origin"
train_path = root_path / "train"
valid_path = root_path / "valid"

In [ ]:
train_origin_files = set([file.stem for file in train_origin_path.rglob("**/*.jpg")])
valid_origin_files = set([file.stem for file in valid_origin_path.rglob("**/*.jpg")])

In [ ]:
assert len(train_origin_files & valid_origin_files) == 0

In [ ]:
import numpy as np
from utils import img_slices_coords,calculate_overlap
index = 0
from collections import defaultdict
train_coll = defaultdict(list)
for img_file in train_origin_path.rglob("**/*.jpg"):
    label = img_file.parent.name
    txt_file = img_file.with_suffix(".txt")
    img = Image.open(img_file)
    slices_coords = img_slices_coords(img, step_size=8)
    bounding_boxes = []
    for line in txt_file.read_text().strip().split("\n"):
        bounding_boxes.append(list(map(float, line.split(" ")[1:])))
    for bounding_box in bounding_boxes:
        for slice_coords in slices_coords:
            ratio = calculate_overlap(img, slice_coords, bounding_box)
            cropped_img = img.crop(slice_coords).resize((512, 512))
            if ratio >.7:
                label_idx = labels.index(label)
                one_hot = np.zeros(len(labels)).astype('float32')
                one_hot[label_idx] = ratio
                one_hot += (1 - ratio) / len(labels)
                train_coll[index].append((np.array(cropped_img), one_hot))
    index += 1


    





In [ ]:
import numpy as np
from utils import img_slices_coords,calculate_overlap
index = 0
from collections import defaultdict
valid_coll = defaultdict(list)
for img_file in valid_origin_path.rglob("**/*.jpg"):
    label = img_file.parent.name
    txt_file = img_file.with_suffix(".txt")
    img = Image.open(img_file)
    slices_coords = img_slices_coords(img, step_size=8)
    bounding_boxes = []
    for line in txt_file.read_text().strip().split("\n"):
        bounding_boxes.append(list(map(float, line.split(" ")[1:])))
    for bounding_box in bounding_boxes:
        for slice_coords in slices_coords:
            ratio = calculate_overlap(img, slice_coords, bounding_box)
            cropped_img = img.crop(slice_coords).resize((512, 512))
            if ratio >.7:
                label_idx = labels.index(label)
                one_hot = np.zeros(len(labels)).astype('float32')
                one_hot[label_idx] = ratio
                one_hot += (1 - ratio) / len(labels)
                valid_coll[index].append((np.array(cropped_img), one_hot))
    index += 1


    





In [ ]:
train_coll

In [ ]:
train_coll.keys()

In [ ]:
train_coll[1][0]

In [ ]:
train_folder = Path("dataset/train")
valid_folder = Path("dataset/valid")

In [ ]:
train_folder.exists()

In [ ]:
valid_folder.exists()

In [ ]:
for k, v in train_coll.items():
    for index, (img, label) in enumerate(v):
        file_name = train_folder / f"{k}_{index}.npz"
        np.savez(file_name, image=img, label=label)

In [ ]:
for k, v in valid_coll.items():
    for index, (img, label) in enumerate(v):
        file_name = valid_folder / f"{k}_{index}.npz"
        np.savez(file_name, image=img, label=label)